In [1]:
#python connect sql
from sqlalchemy import create_engine, text
import psycopg2
import psycopg2.extras
import pandas as pd
import json
import os

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host = db_conn_dict['host']
        db_user = db_conn_dict['user']
        db_pw = db_conn_dict['password']
        default_db = db_conn_dict['database']
    try:
        db = create_engine('postgresql+psycopg2://' + db_user + ':' + db_pw + '@' + host + '/' + default_db, echo=False)
        conn = db.connect()
        conn.autocommit = True
        print('Connected successfully.')
    except Exception as e:
        print("Unable to connect to the database.")
        print(e)
        db, conn = None, None
    return db, conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(sqlcmd, args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

db, conn = pgconnect(credentials)

Connected successfully.


In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt
import numpy as np

srid = 4326
SA2 = gpd.read_file("SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp")
SA2 = SA2[SA2['GCC_NAME21'] == 'Greater Sydney']

def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)

SA2['geometry_wkt'] = SA2['geometry'].apply(lambda x: create_wkt_element(x, srid=srid))
SA2 = SA2.drop(columns=['GCC_CODE21', 'GCC_NAME21', 'STE_CODE21', 'STE_NAME21', 'AUS_CODE21', 'AUS_NAME21', 'geometry'])


In [3]:
conn.execute("""CREATE EXTENSION IF NOT EXISTS postgis;
""")

In [4]:
conn.execute("""
DROP TABLE IF EXISTS "sa2";
CREATE TABLE "sa2" (
    "sa2_code21" INTEGER PRIMARY KEY,
    "sa2_name21" VARCHAR(50),
    "chg_flag21" INTEGER,
    "chg_lbl21" VARCHAR(20),
    "sa3_code21" INTEGER,
    "sa3_name21" VARCHAR(50),
    "sa4_code21" INTEGER,
    "sa4_name21" VARCHAR(50),
    "areasqkm21" FLOAT,
    "loci_uri21" VARCHAR(255),
    geom GEOMETRY(MULTIPOLYGON, 4326)
);
""")

In [5]:
SA2.to_sql("sa2", conn, if_exists='replace', index=False, dtype={
    'geometry_wkt': Geometry('MULTIPOLYGON', srid=4326)
})
query(conn, 'SELECT * FROM "sa2"').head(1)

SA2_CODE21                SA2_NAME21 CHG_FLAG21  CHG_LBL21 SA3_CODE21  \
0  102011028  Avoca Beach - Copacabana          0  No change      10201   

  SA3_NAME21 SA4_CODE21     SA4_NAME21  AREASQKM21  \
0    Gosford        102  Central Coast      6.4376   

                                          LOCI_URI21  \
0  http://linked.data.gov.au/dataset/asgsed3/SA2/...   

                                        geometry_wkt  
0  0106000020E6100000010000000103000000010000005E...

In [6]:
businesses = pd.read_csv('Businesses.csv')

In [7]:
conn.execute("""
DROP TABLE IF EXISTS businesses;
CREATE TABLE businesses (
    "industry_code" VARCHAR(5) PRIMARY KEY,
    "industry_name" VARCHAR(50),
    "sa2_code" INTEGER,
    "sa2_name" VARCHAR(50),
    "0_to_50k_businesses" INTEGER,
    "50k_to_200k_businesses" INTEGER,
    "200k_to_2m_businesses" INTEGER,
    "2m_to_5m_businesses" INTEGER,
    "5m_to_10m_businesses" INTEGER,
    "10m_or_more_businesses" INTEGER,
    total_businesses INTEGER
);
""")

In [8]:
businesses.to_sql("businesses", conn, if_exists='replace', index=False)
query(conn, 'SELECT * FROM "businesses"').head(1)

industry_code                      industry_name   sa2_code   sa2_name  \
0             A  Agriculture, Forestry and Fishing  101021007  Braidwood   

   0_to_50k_businesses  50k_to_200k_businesses  200k_to_2m_businesses  \
0                  136                      92                     63   

   2m_to_5m_businesses  5m_to_10m_businesses  10m_or_more_businesses  \
0                    4                     0                       0   

   total_businesses  
0               296

In [9]:
income = pd.read_csv('Income.csv')

cols_to_convert = ['earners', 'median_age', 'median_income', 'mean_income']
income[cols_to_convert] = income[cols_to_convert].replace('np', np.nan)
for col in cols_to_convert:
    income[col] = pd.to_numeric(income[col], errors='coerce')
    median_val = income[col].median()
    income[col] = income[col].fillna(median_val).astype(int) 
income.dtypes

sa2_code21        int64
sa2_name         object
earners           int64
median_age        int64
median_income     int64
mean_income       int64
dtype: object

In [10]:
conn.execute("""
DROP TABLE IF EXISTS income;
CREATE TABLE income (
    "sa2_code21" INTEGER PRIMARY KEY,
    "sa2_name" VARCHAR(50),
    "earners" INTEGER,
    "median_age" INTEGER,
    "median_income" INTEGER,
    "mean_income" INTEGER
);
""")

In [11]:
income.to_sql("income", conn, if_exists='replace', index=False)
query(conn, 'SELECT * FROM "income"').head(1)

sa2_code21   sa2_name  earners  median_age  median_income  mean_income
0   101021007  Braidwood     2467          51          46640        68904

In [12]:
stops = pd.read_csv('Stops.txt', quotechar='"')
stops['geometry'] = gpd.points_from_xy(stops['stop_lon'], stops['stop_lat'])
stops.head(1)

stop_id  stop_code                          stop_name   stop_lat  \
0  200039   200039.0  Central Station, Eddy Av, Stand A -33.882206   

     stop_lon  location_type parent_station  wheelchair_boarding  \
0  151.206665            NaN         200060                    0   

  platform_code                     geometry  
0           NaN  POINT (151.20666 -33.88221)

In [13]:
conn.execute("""
DROP TABLE IF EXISTS stops;
CREATE TABLE stops (
    stop_id VARCHAR(20) PRIMARY KEY,
    stop_code VARCHAR(20),
    stop_name VARCHAR(255),
    stop_lat VARCHAR(255),
    stop_lon VARCHAR(255),
    location_type VARCHAR(255),
    parent_station VARCHAR(255),
    wheelchair_boarding VARCHAR(255),
    platform_code VARCHAR(255),
    geometry_wkt GEOMETRY(POINT,4326)
);
""")

In [14]:
def create_wkt_element(geom, srid=4326):
    return WKTElement(geom.wkt, srid)
stops['geometry_wkt'] = stops['geometry'].apply(lambda x: create_wkt_element(x, srid=srid))
stops = stops.drop(columns=['geometry'])
stops.to_sql('stops', conn, if_exists='replace', index=False, dtype={'geometry_wkt': Geometry('POINT', srid=4326)})
query(conn, 'SELECT * FROM "stops"').head(1)

stop_id  stop_code                          stop_name   stop_lat  \
0  200039   200039.0  Central Station, Eddy Av, Stand A -33.882206   

     stop_lon  location_type parent_station  wheelchair_boarding  \
0  151.206665            NaN         200060                    0   

  platform_code                                       geometry_wkt  
0          None  0101000020E6100000FFA631FF9CE66240A1FF6524ECF0...

In [15]:
population = pd.read_csv('Population.csv')

In [16]:
conn.execute("""
DROP TABLE IF EXISTS population;
CREATE TABLE population (
    "sa2_code" INTEGER PRIMARY KEY,
    "sa2_name" VARCHAR(50),
    "0-4_people" INTEGER,
    "5-9_people" INTEGER,
    "10-14_people" INTEGER,
    "15-19_people" INTEGER,
    "20-24_people" INTEGER,
    "25-29_people" INTEGER,
    "30-34_people" INTEGER,
    "35-39_people" INTEGER,
    "40-44_people" INTEGER,
    "45-49_people" INTEGER,
    "50-54_people" INTEGER,
    "55-59_people" INTEGER,
    "60-64_people" INTEGER,
    "65-69_people" INTEGER,
    "70-74_people" INTEGER,
    "75-79_people" INTEGER,
    "80-84_people" INTEGER,
    "85-and-over_people" INTEGER,
    "total_people" INTEGER
);
""")


In [17]:
population.to_sql("population", conn, if_exists='replace', index=False)
query(conn, 'SELECT * FROM "population"').head(1)

sa2_code                  sa2_name  0-4_people  5-9_people  10-14_people  \
0  102011028  Avoca Beach - Copacabana         424         522           623   

   15-19_people  20-24_people  25-29_people  30-34_people  35-39_people  ...  \
0           552           386           222           306           416  ...   

   45-49_people  50-54_people  55-59_people  60-64_people  65-69_people  \
0           572           602           570           520           464   

   70-74_people  75-79_people  80-84_people  85-and-over_people  total_people  
0           369           226           142                  70          7530  

[1 rows x 21 columns]

In [18]:
pollingplaces2019 = pd.read_csv('PollingPlaces2019.csv')
pollingplaces2019['the_geom'] = gpd.points_from_xy(pollingplaces2019.longitude, pollingplaces2019.latitude)
pollingplaces2019['geometry_wkt'] = pollingplaces2019['the_geom'].apply(lambda x: create_wkt_element(geom=x, srid=srid)) 
pollingplaces2019 = pollingplaces2019.drop(columns=['premises_state_abbreviation', 'the_geom'])

In [19]:
conn.execute("""
DROP TABLE IF EXISTS pollingplaces2019;
CREATE TABLE pollingplaces2019 (
    "FID" VARCHAR(100),
    "state" VARCHAR(10),
    "division_id" INTEGER,
    "division_name" VARCHAR(10),
    "polling_place_id" INTEGER PRIMARY KEY,
    "polling_place_type_id" INTEGER,
    "polling_place_name" VARCHAR(50),
    "premises_name" VARCHAR(50),
    "premises_address_1" VARCHAR(50),
    "premises_address_2" VARCHAR(50),
    "premises_address_3" VARCHAR(50),
    "premises_suburb" VARCHAR(50),
    "premises_post_code" INTEGER,
    "latitude" FLOAT,
    "longitude" FLOAT,
    geometry_wkt GEOMETRY(MULTIPOLYGON,4326)
);
""")

In [20]:
pollingplaces2019.to_sql('pollingplaces2019', conn, if_exists='replace', index=False, dtype={'geometry_wkt': Geometry('POINT', srid=4326)})
query(conn, 'SELECT * FROM "pollingplaces2019"').head(1)

FID state  division_id  \
0  aec_federal_election_polling_places_2019.fid-4...   NSW          104   

  division_name  polling_place_id  polling_place_type_id  \
0        Barton             33595                      2   

        polling_place_name   premises_name premises_address_1  \
0  Special Hospital Team 1  Multiple sites               None   

  premises_address_2 premises_address_3 premises_suburb  premises_post_code  \
0               None               None            None                 NaN   

   latitude  longitude                                       geometry_wkt  
0       NaN        NaN  0101000020E6100000000000000000F87F000000000000...

In [21]:
def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)
catchments_future = gpd.read_file("catchments/catchments_future.shp")
catchments_future['geometry_wkt'] = catchments_future['geometry'].apply(lambda x: create_wkt_element(geom=x, srid=srid)) 
catchments_future = catchments_future.drop(columns="geometry")
catchments_future.head(1)

USE_ID CATCH_TYPE        USE_DESC  ADD_DATE  KINDERGART  YEAR1  YEAR2  \
0   8416  HIGH_COED  Ku-ring-gai HS  20230114           0      0      0   

   YEAR3  YEAR4  YEAR5  YEAR6  YEAR7  YEAR8  YEAR9  YEAR10  YEAR11  YEAR12  \
0      0      0      0      0   2024   2024   2024    2024    2024    2024   

                                        geometry_wkt  
0  MULTIPOLYGON (((151.19848917708944 -33.5398987...

In [22]:
catchments_primary = gpd.read_file("catchments/catchments_primary.shp")
catchments_primary['geometry_wkt'] = catchments_primary['geometry'].apply(lambda x: create_wkt_element(geom=x, srid=srid)) 
catchments_primary = catchments_primary.drop(columns="geometry")
catchments_primary.head(1)

USE_ID CATCH_TYPE    USE_DESC  ADD_DATE KINDERGART YEAR1 YEAR2 YEAR3 YEAR4  \
0   2838    PRIMARY  Parklea PS  20181210          Y     Y     Y     Y     Y   

  YEAR5 YEAR6 YEAR7 YEAR8 YEAR9 YEAR10 YEAR11 YEAR12 PRIORITY  \
0     Y     Y     N     N     N      N      N      N     None   

                                        geometry_wkt  
0  MULTIPOLYGON (((150.93563850416004 -33.7161211...

In [23]:
catchments_secondary = gpd.read_file("catchments/catchments_secondary.shp")
catchments_secondary['geometry_wkt'] = catchments_secondary['geometry'].apply(lambda x: create_wkt_element(geom=x, srid=srid)) 
catchments_secondary = catchments_secondary.drop(columns="geometry")
catchments_secondary.head(1)

USE_ID CATCH_TYPE      USE_DESC  ADD_DATE KINDERGART YEAR1 YEAR2 YEAR3  \
0   8503  HIGH_COED  Billabong HS  20200507          N     N     N     N   

  YEAR4 YEAR5 YEAR6 YEAR7 YEAR8 YEAR9 YEAR10 YEAR11 YEAR12 PRIORITY  \
0     N     N     N     Y     Y     Y      Y      Y      Y     None   

                                        geometry_wkt  
0  MULTIPOLYGON (((146.67182402032344 -35.3144375...

In [24]:
conn.execute("""

DROP TABLE IF EXISTS catchments_future;
CREATE TABLE catchments_future (
    "USE_ID" VARCHAR(255),
    "CATCH_TYPE" VARCHAR(255),
    "USE_DESC" VARCHAR(255),
    "ADD_DATE" DATE,
    "KINDERGART" INTEGER,
    "YEAR1" INTEGER,
    "YEAR2" INTEGER,
    "YEAR3" INTEGER,
    "YEAR4" INTEGER,
    "YEAR5" INTEGER,
    "YEAR6" INTEGER,
    "YEAR7" INTEGER,
    "YEAR8" INTEGER,
    "YEAR9" INTEGER,
    "YEAR10" INTEGER,
    "YEAR11" INTEGER,
    "YEAR12" INTEGER,
    geometry_wkt GEOMETRY(MULTIPOLYGON,4326)
);
""")

conn.execute("""
DROP TABLE IF EXISTS catchments_primary;
CREATE TABLE catchments_primary (
    "USE_ID" VARCHAR(255),
    "CATCH_TYPE" VARCHAR(255),
    "USE_DESC" VARCHAR(255),
    "ADD_DATE" VARCHAR(255),
    "KINDERGART" VARCHAR(1),
    "YEAR1" VARCHAR(1),
    "YEAR2" VARCHAR(1),
    "YEAR3" VARCHAR(1),
    "YEAR4" VARCHAR(1),
    "YEAR5" VARCHAR(1),
    "YEAR6" VARCHAR(1),
    "YEAR7" VARCHAR(1),
    "YEAR8" VARCHAR(1),
    "YEAR9" VARCHAR(1),
    "YEAR10" VARCHAR(1),
    "YEAR11" VARCHAR(1),
    "YEAR12" VARCHAR(1),
    "PRIORITY" VARCHAR(255),
    geometry_wkt GEOMETRY(MULTIPOLYGON,4326)
);
""")

conn.execute("""
DROP TABLE IF EXISTS catchments_secondary;
CREATE TABLE catchments_secondary (
    "USE_ID" VARCHAR(255),
    "CATCH_TYPE" VARCHAR(255),
    "USE_DESC" VARCHAR(255),
    "ADD_DATE" VARCHAR(255),
    "KINDERGART" VARCHAR(1),
    "YEAR1" VARCHAR(1),
    "YEAR2" VARCHAR(1),
    "YEAR3" VARCHAR(1),
    "YEAR4" VARCHAR(1),
    "YEAR5" VARCHAR(1),
    "YEAR6" VARCHAR(1),
    "YEAR7" VARCHAR(1),
    "YEAR8" VARCHAR(1),
    "YEAR9" VARCHAR(1),
    "YEAR10" VARCHAR(1),
    "YEAR11" VARCHAR(1),
    "YEAR12" VARCHAR(1),
    "PRIORITY" VARCHAR(255),
    geometry_wkt GEOMETRY(MULTIPOLYGON,4326)
);
""")

In [25]:
catchments_future.dtypes

USE_ID          object
CATCH_TYPE      object
USE_DESC        object
ADD_DATE        object
KINDERGART       int32
YEAR1            int32
YEAR2            int32
YEAR3            int32
YEAR4            int32
YEAR5            int32
YEAR6            int32
YEAR7            int32
YEAR8            int32
YEAR9            int32
YEAR10           int32
YEAR11           int32
YEAR12           int32
geometry_wkt    object
dtype: object

In [26]:
catchments_future.to_sql("catchments_future", conn, if_exists='replace', index=False, dtype={
    'geometry_wkt': Geometry('MULTIPOLYGON', srid)
})
catchments_primary.to_sql("catchments_primary", conn, if_exists='replace', index=False, dtype={
    'geometry_wkt': Geometry('MULTIPOLYGON', srid)
})
catchments_secondary.to_sql("catchments_secondary", conn, if_exists='replace', index=False, dtype={
    'geometry_wkt': Geometry('MULTIPOLYGON', srid)
})

436

In [27]:
total_catchments = pd.concat([catchments_primary, catchments_secondary, catchments_future])
total_catchments.head(1)

USE_ID CATCH_TYPE    USE_DESC  ADD_DATE KINDERGART YEAR1 YEAR2 YEAR3 YEAR4  \
0   2838    PRIMARY  Parklea PS  20181210          Y     Y     Y     Y     Y   

  YEAR5 YEAR6 YEAR7 YEAR8 YEAR9 YEAR10 YEAR11 YEAR12 PRIORITY  \
0     Y     Y     N     N     N      N      N      N     None   

                                        geometry_wkt  
0  MULTIPOLYGON (((150.93563850416004 -33.7161211...

In [28]:
total_catchments.to_sql("total_catchments", conn, if_exists='replace', index=False, dtype={
    'geometry_wkt': Geometry('MULTIPOLYGON', srid)
})

128